# Reference

To run this code you will need to install [Matplotlib](https://matplotlib.org/users/installing.html) and [Numpy](https://www.scipy.org/install.html)

If you like to run the example locally follow the instructions provided on [Keras website](https://keras.io/#installation)

It's __strongly__ suggested to use a Python environments manager such as [Conda](https://conda.io/docs/) or some kind of [VirutalEnv](#)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/digitalideation/digcre_h2301/blob/master/samples/week02/03-shape-example.ipynb)

---

Downgrade Matplotlib

In [ ]:
!pip install matplotlib==3.7.0

# A second look at a neural network

Let's try to adapt the shape classification model built with the `toyNN` in js before.

We first need to create a dataset _manually_ to do we will define a `draw_shape` function that will help generating some random shape

In [ ]:
%matplotlib inline
import math
import random
import matplotlib.pyplot as plt
import numpy as np

# 0 = rectangle, 1 = triangle, 2 = ellipse
# return shape
def draw_shape(max_size, type):

    # Not so random size and random coordinate
    s = int(max_size/2 + random.random() * max_size/2)
    x = int(s/2) + random.random() * (max_size-s)
    y = int(s/2) + random.random() * (max_size-s)

    type = type%3
    
    if type == 0:
        art = plt.Rectangle((x-s/2, y-s/2), s, s, color='r')

    if type == 1:
        verts = [
            (x-s/2, y-s/2),
            (x, y+s/2),
            (x+s/2, y-s/2)
        ]
        art = plt.Polygon(verts, color='r')

    if type == 2:
        art = plt.Circle((x, y), s/2, color='r')
    
    return art

We also define a helper function that convert a matplotlib figure to a np array

In [ ]:
# https://stackoverflow.com/a/7821917
def fig2rgb_array(fig):
    fig.canvas.draw()
    buf = fig.canvas.tostring_rgb()
    ncols, nrows = fig.canvas.get_width_height()
    return np.frombuffer(buf, dtype=np.uint8).reshape(nrows, ncols, 3)

Let's test the function see if it works as expected

In [ ]:
# Image and dataset size we are going to use
image_size = 48.0
dataset_size = 500

# Create plot's figure and axes
# https://stackoverflow.com/a/638443
fig = plt.figure(figsize=(1,1), dpi=image_size)
ax = fig.add_subplot(111)

# Setting for the axes
ax.set_xlim(0,image_size)
ax.set_ylim(0,image_size)
# ax.axis('off')

# Draw a random shape
art = draw_shape(image_size,random.randint(0,2))
# Add the shape to the plot
# https://stackoverflow.com/a/29184075
plt.gcf().gca().add_artist(art)
# gcf() means Get Current Figure
# gca() means Get Current Axis

# convert the figure to an array
data = fig2rgb_array(fig)
print(data.shape)

Let's create a loop that will generate a small dataset for us

In [ ]:
def generate_dataset(image_size, dataset_size):

    # Those variable will contain the images and associated labels
    images = np.zeros((dataset_size, image_size, image_size, 3))
    labels = np.zeros((dataset_size))
    
    # The plot figure we will use to generate the shapes
    fig = plt.figure(figsize=(1,1), dpi=image_size)

    for i in range(dataset_size):
        
        # Clear the figure
        fig.clf()
        
        # Recreate the axes
        ax = fig.add_subplot(111)
        ax.set_xlim(0, image_size)
        ax.set_ylim(0, image_size)
        ax.axis('off')
        
        # Define label
        label = i%3
        art = draw_shape(image_size, label)
        plt.gcf().gca().add_artist(art)
        
        # Add values to the arrays
        images[i] = fig2rgb_array(fig)
        labels[i] = label
        
    return images, labels

# Generate our dataset
images, labels = generate_dataset(int(image_size), dataset_size)
print(images.shape)
print(labels.shape)

Eventually we can save our dataset for later, since it takes quite some time to generate it 😉

In [ ]:
np.save('datasets/shape-example-shapes1.npy', images)
np.save('datasets/shape-example-labels1.npy', labels)

If we need to load it we can then use the following code

In [ ]:
images = np.load('datasets/shape-example-shapes1.npy')
labels = np.load('datasets/shape-example-labels1.npy')

We split our dataset manually in training and testing set

In [ ]:
# Define the size of the training set, here we use 80% of the total samples for training
train_size = int(dataset_size*.8)

# TODO: We should shuffle the dataset

# Split the dataset into train and test dataset
train_images, test_images = images[:train_size], images[train_size:]
train_labels, test_labels = labels[:train_size], labels[train_size:]

# Verify the data
print(train_images.shape)
print(train_labels.shape)

# sample_images = []
# for label, image in list(zip(train_labels, train_images))[:10]:
#     fig1, ax1 = plt.subplots()
#     ax1.axis('off')
#     plt.title(label)
#     fig1.add_subplot(111).imshow(image/255)

full_image = np.concatenate(train_images[:12]/255, axis=1)
plt.figure(figsize=(16,4))
plt.imshow(full_image)

Now we can create our model

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
model = keras.Sequential([
    layers.Dense(512, activation="sigmoid"),
    layers.Dense(3, activation="softmax")
])

And compile it

In [ ]:
model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

Before training, we will preprocess our data by reshaping it into the shape that the network expects, and scaling it so that all values are in the `[0, 1]` interval. Then we also need to categorically encode the labels.

In [ ]:
# Reshape data
image_size = int(image_size)
train_images = train_images.reshape((len(train_images), 3 * image_size * image_size))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((len(test_images), 3 * image_size * image_size))
test_images = test_images.astype('float32') / 255

# Encode to categorical
from tensorflow.keras.utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

Then we can start the training

In [ ]:
model.fit(train_images, train_labels, epochs=100, batch_size=128)

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

In [ ]:
print('test_acc:', test_acc)

Show images which were wrongly predicted

In [ ]:
for label, image in list(zip(test_labels, test_images)):
    prediction = model.predict(np.array([image,]))
    if not prediction.argmax() == label.argmax():
        image = image.reshape(48, 48, 3)
        fig1, ax1 = plt.subplots()
        ax1.axis('off')
        plt.title('predicted:' + str(prediction.argmax()))
        fig1.add_subplot(111).imshow(image)    